In [2]:
import sys
print(sys.executable)


C:\Users\Raji_kabilan\miniconda3\envs\exoplanet\python.exe


In [3]:
# Install required packages
!pip install scikit-learn==1.4.2
!pip install imbalanced-learn==0.12.4



  Using cached scikit_learn-1.4.2-cp311-cp311-win_amd64.whl.metadata (11 kB)
Using cached scikit_learn-1.4.2-cp311-cp311-win_amd64.whl (10.6 MB)
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.3.2
    Uninstalling scikit-learn-1.3.2:
      Successfully uninstalled scikit-learn-1.3.2


  You can safely remove it manually.
  You can safely remove it manually.
  You can safely remove it manually.


  Using cached imbalanced_learn-0.12.4-py3-none-any.whl.metadata (8.3 kB)
Using cached imbalanced_learn-0.12.4-py3-none-any.whl (258 kB)
  Attempting uninstall: imbalanced-learn
    Found existing installation: imbalanced-learn 0.11.0
    Uninstalling imbalanced-learn-0.11.0:
      Successfully uninstalled imbalanced-learn-0.11.0


In [30]:
import pandas as pd
import numpy as np

df_clean = pd.read_csv('./Dataset/phl_exoplanet_after_basic_eda.csv')



In [31]:
num_cols = df_clean.select_dtypes(include=['int64', 'float64']).columns
num_cols = num_cols.drop('P_HABITABLE', errors='ignore')



In [32]:
Q1 = df_clean[num_cols].quantile(0.25)
Q3 = df_clean[num_cols].quantile(0.75)
IQR = Q3 - Q1

outlier_mask = (
    (df_clean[num_cols] < (Q1 - 1.5 * IQR)) |
    (df_clean[num_cols] > (Q3 + 1.5 * IQR))
)

outlier_counts = outlier_mask.sum().sort_values(ascending=False)
outlier_counts


P_PERIOD_ERROR_MIN              798
P_PERIOD_ERROR_MAX              794
P_MASS_EST                      715
P_APASTRON                      695
P_PERIOD                        691
                               ... 
P_IMPACT_PARAMETER_ERROR_MAX      5
P_RADIUS_EST                      3
P_IMPACT_PARAMETER                2
S_AGE_ERROR_MAX                   1
P_STATUS                          0
Length: 72, dtype: int64

In [33]:

# IQR Capping
df_outlier_fixed = df_clean.copy()

for col in num_cols:
    lower = Q1[col] - 1.5 * IQR[col]
    upper = Q3[col] + 1.5 * IQR[col]
    df_outlier_fixed[col] = df_outlier_fixed[col].clip(lower, upper)



In [34]:
skewness = df_outlier_fixed[num_cols].skew().sort_values(ascending=False)
skewness.head(10)


P_HILL_SPHERE                  1.270102
S_DISTANCE_ERROR_MAX           1.244766
P_MASS_EST                     1.230766
S_RADIUS_ERROR_MAX             1.229350
P_SEMI_MAJOR_AXIS_ERROR_MAX    1.229038
P_MASS_ERROR_MAX               1.216073
P_FLUX_MAX                     1.211727
P_SEMI_MAJOR_AXIS              1.210902
P_FLUX                         1.206222
P_ANGULAR_DISTANCE             1.203781
dtype: float64

In [35]:

skewed_cols = skewness[skewness > 1].index

for col in skewed_cols:
    df_outlier_fixed[col] = np.log1p(df_outlier_fixed[col])



In [36]:
df_outlier_fixed.to_csv(
    './Dataset/phl_exoplanet_after_outliers.csv',
    index=False
)



In [37]:
cat_cols = df_outlier_fixed.select_dtypes(include=['object']).columns



In [38]:
df_cat_clean = df_outlier_fixed.copy()

for col in cat_cols:
    df_cat_clean[col] = (
        df_cat_clean[col]
        .astype(str)
        .str.strip()
        .str.upper()
    )



In [39]:
y = df_cat_clean['P_HABITABLE']
X = df_cat_clean.drop(columns=['P_HABITABLE'])



In [40]:
# One-Hot Encode
X_encoded = pd.get_dummies(X, columns=cat_cols, drop_first=False)

# Identify numeric and dummy (0/1) columns
num_cols_after = X_encoded.select_dtypes(include=['int64','float64']).columns
dummy_cols = [c for c in X_encoded.columns if c not in num_cols_after]

# Impute numeric features with median
for col in num_cols_after:
    median_val = X_encoded[col].median()
    X_encoded[col] = X_encoded[col].fillna(median_val)

# Impute dummy (categorical) features with mode
for col in dummy_cols:
    mode_val = X_encoded[col].mode()[0]
    X_encoded[col] = X_encoded[col].fillna(mode_val)

# Confirm no remaining NaNs
print("Remaining NaNs after final imputation:", X_encoded.isna().sum().sum())

Remaining NaNs after final imputation: 0


In [41]:
X_encoded.shape

(4048, 14422)

In [42]:
X_encoded.head()

,P_STATUS,P_MASS,P_MASS_ERROR_MIN,P_MASS_ERROR_MAX,P_RADIUS,P_RADIUS_ERROR_MIN,P_RADIUS_ERROR_MAX,P_YEAR,P_PERIOD,P_PERIOD_ERROR_MIN,...,S_CONSTELLATION_ENG_TOUCAN,S_CONSTELLATION_ENG_TRIANGLE,S_CONSTELLATION_ENG_TWINS,S_CONSTELLATION_ENG_UNICORN,S_CONSTELLATION_ENG_VIRGIN,S_CONSTELLATION_ENG_WATER CARRIER,S_CONSTELLATION_ENG_WATER SNAKE,S_CONSTELLATION_ENG_WHALE,S_CONSTELLATION_ENG_WINGED HORSE,S_CONSTELLATION_ENG_WOLF
0,3.0,7.589537,-192.053928,5.341662,2.33168,-0.23541,0.28148,2011,4.594317,-0.002799,...,False,False,False,False,False,False,False,False,False,False
1,3.0,7.589537,-192.053928,5.341662,2.33168,-0.23541,0.28148,2011,4.594317,-0.002799,...,False,False,False,False,False,False,False,False,False,False
2,3.0,7.330782,-24.154928,3.262258,2.33168,-0.23541,0.28148,2011,4.594317,-0.002799,...,False,False,False,False,False,False,False,False,False,False
3,3.0,7.301201,-47.674200,3.885149,2.33168,-0.23541,0.28148,2011,4.594317,-0.002799,...,False,False,False,False,False,False,False,False,False,False
4,3.0,6.339890,-25.426240,3.274357,2.33168,-0.23541,0.28148,2011,4.594317,-0.002799,...,False,False,False,False,False,False,False,False,False,False


In [43]:
num_cols = X_encoded.select_dtypes(include=['int64', 'float64']).columns
num_cols

Index(['P_STATUS', 'P_MASS', 'P_MASS_ERROR_MIN', 'P_MASS_ERROR_MAX',
       'P_RADIUS', 'P_RADIUS_ERROR_MIN', 'P_RADIUS_ERROR_MAX', 'P_YEAR',
       'P_PERIOD', 'P_PERIOD_ERROR_MIN', 'P_PERIOD_ERROR_MAX',
       'P_SEMI_MAJOR_AXIS', 'P_SEMI_MAJOR_AXIS_ERROR_MIN',
       'P_SEMI_MAJOR_AXIS_ERROR_MAX', 'P_ECCENTRICITY', 'P_ANGULAR_DISTANCE',
       'P_IMPACT_PARAMETER', 'P_IMPACT_PARAMETER_ERROR_MIN',
       'P_IMPACT_PARAMETER_ERROR_MAX', 'S_RA', 'S_DEC', 'S_MAG', 'S_DISTANCE',
       'S_DISTANCE_ERROR_MIN', 'S_DISTANCE_ERROR_MAX', 'S_METALLICITY',
       'S_METALLICITY_ERROR_MIN', 'S_METALLICITY_ERROR_MAX', 'S_MASS',
       'S_MASS_ERROR_MIN', 'S_MASS_ERROR_MAX', 'S_RADIUS',
       'S_RADIUS_ERROR_MIN', 'S_RADIUS_ERROR_MAX', 'S_AGE', 'S_AGE_ERROR_MIN',
       'S_AGE_ERROR_MAX', 'S_TEMPERATURE', 'S_TEMPERATURE_ERROR_MIN',
       'S_TEMPERATURE_ERROR_MAX', 'S_LOG_G', 'P_HILL_SPHERE', 'P_DISTANCE',
       'P_PERIASTRON', 'P_APASTRON', 'P_DISTANCE_EFF', 'P_FLUX', 'P_FLUX_MIN',
       'P_FL

In [44]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

X_scaled_array = scaler.fit_transform(X_encoded[num_cols])

In [45]:
X_scaled = pd.DataFrame(
    X_scaled_array,
    columns=num_cols,
    index=X_encoded.index
)

In [46]:
X_final = X_scaled
X_final.describe().T[['mean', 'std']].head()

,mean,std
P_STATUS,0.000000e+00,0.000000
P_MASS,-8.504396e-16,1.000124
P_MASS_ERROR_MIN,-3.510587e-17,1.000124
P_MASS_ERROR_MAX,-2.878681e-16,1.000124
P_RADIUS,-7.021173e-17,1.000124


In [47]:
final_df = X_final.copy()
final_df['P_HABITABLE'] = y.values

final_df.to_csv(
    './Dataset/phl_exoplanet_preprocessed_no_sampling.csv',
    index=False
)

In [48]:
X_final.isna().sum().sum()

0

In [49]:
nan_cols = X_final.columns[X_final.isna().any()]
nan_cols

Index([], dtype='object')

In [50]:
X_final[nan_cols].isna().sum().sort_values(ascending=False)

Series([], dtype: float64)